In [1]:
from sql_connector import mydb, mycursor
import numpy as np
import pandas as pd
import random


# Set seed for reproducibility
np.random.seed(42)
random.seed(42)



In [2]:
# Extract all retweets
mycursor.execute("""select *
                    FROM en_retweets
                    where original_tweet_text is not null
                    """)
retweets_all = mycursor.fetchall()
retweets_all = pd.DataFrame(retweets_all, columns=[desc[0] for desc in mycursor.description])
print("all retweets: ", retweets_all.shape)

all retweets:  (448519, 53)


In [3]:
# Add topic modeling
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', download_dir=r'C:\\Users\\pelegel\\PycharmProjects\\pythonProject1\\myenv\\nltk_data')
nltk.download('punkt_tab', download_dir=r'C:\\Users\\pelegel\\PycharmProjects\\pythonProject1\\myenv\\nltk_data')

def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'@\w+|\#\w+', '', text)  # Remove mentions and hashtags
    text = re.sub(r'\W|\d', ' ', text)  # Remove special characters and digits
    text = text.lower()    # Convert to lowercase
    tokens = word_tokenize(text)   # Tokenize
    lemmatizer = WordNetLemmatizer()    # Initialize lemmatizer
    # Remove stopwords and specific words, then lemmatize
    stop_words = set(stopwords.words('english') + ['chatgpt', 'gpt', 'chat gpt', 'chat'])
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and len(token) > 2]
    return ' '.join(tokens)



[nltk_data] Downloading package punkt to C:\\Users\\pelegel\\PycharmPr
[nltk_data]     ojects\\pythonProject1\\myenv\\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\\Users\\pelegel\\Pycha
[nltk_data]     rmProjects\\pythonProject1\\myenv\\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
# Add topic modeling
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', download_dir=r'C:\\Users\\pelegel\\PycharmProjects\\pythonProject1\\myenv\\nltk_data')
nltk.download('punkt_tab', download_dir=r'C:\\Users\\pelegel\\PycharmProjects\\pythonProject1\\myenv\\nltk_data')
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

retweets_all['processed_text'] = retweets_all['original_tweet_text'].apply(preprocess_text)

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=1000, max_df=0.95, min_df=2)
tfidf = vectorizer.fit_transform(retweets_all['processed_text'])

# Apply NMF
num_topics = 9
nmf_model = NMF(n_components=num_topics, random_state=42)
nmf_output = nmf_model.fit_transform(tfidf)

# Get the top words for each topic
feature_names = vectorizer.get_feature_names_out()
word_dict = {}
for i in range(num_topics):
    words_ids = nmf_model.components_[i].argsort()[:-10 - 1:-1]
    words = [feature_names[key] for key in words_ids]
    word_dict[f'Topic {i+1}'] = words

# Print the topics and their top words
for topic, words in word_dict.items():
    print(f"{topic}: {', '.join(words)}")

# Assign topics to tweets
retweets_all['dominant_topic'] = nmf_output.argmax(axis=1)
retweets_all['dominant_topic'] = retweets_all['dominant_topic'] + 1
print(retweets_all['dominant_topic'])
one_hot_encoded = pd.get_dummies(retweets_all['dominant_topic'], prefix='topic')
retweets_all = pd.concat([retweets_all, one_hot_encoded], axis=1)

[nltk_data] Downloading package punkt to C:\\Users\\pelegel\\PycharmPr
[nltk_data]     ojects\\pythonProject1\\myenv\\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\\Users\\pelegel\\Pycha
[nltk_data]     rmProjects\\pythonProject1\\myenv\\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Topic 1: month, user, netflix, took, instagram, crossed, compare, day, year, million
Topic 2: thought, architecture, solve, implemented, blockchain, unique, open, asked, write, answer
Topic 3: use, google, amp, guide, full, created, bring, potential, ad, figure
Topic 4: one, ago, whisper, tool, copilot, codex, github, instructgpt, exist, slide
Topic 5: done, nothing, week, senior, pov, asks, frantically, engineer, elon, data
Topic 6: today, game, thing, real, want, system, share, please, changing, thr
Topic 7: new, far, left, york, justice, cause, social, rename, time, good
Topic 8: offer, fossil, fuel, argument, expressly, prohibits, alarm, used, prohibited, promoting
Topic 9: email, addy, write, introducing, powered, assistant, done, asked, story, using
0         3
1         3
2         3
3         9
4         3
         ..
448514    6
448515    1
448516    6
448517    2
448518    6
Name: dominant_topic, Length: 448519, dtype: int64


In [20]:
retweets_all = retweets_all.drop(['dominant_topic', 'processed_text'], axis=1)

KeyError: "['dominant_topic', 'processed_text'] not found in axis"

In [5]:
# Extract users who first retweeted and then tweeted -> Class 1
mycursor.execute("""
                    select u.*
                    FROM en_users u
                    JOIN (SELECT author_id, MIN(created_at) AS earliest_retweet FROM en_retweets GROUP BY author_id) r ON u.id = r.author_id
                    JOIN (SELECT author_id, MIN(created_at) AS earliest_tweet FROM en_tweets GROUP BY author_id) t ON u.id = t.author_id
                    WHERE t.earliest_tweet > r.earliest_retweet;
                    """)
users_t = mycursor.fetchall()
users_t = pd.DataFrame(users_t, columns=[desc[0] for desc in mycursor.description])
print("users class 1 :", users_t.shape)

# Extract users who retweeted but didn't tweeted at all -> Class 0
mycursor.execute("""
                    SELECT u.*
                    FROM en_retweets r join en_users u on r.author_id=u.id
                    WHERE author_id NOT IN (SELECT author_id FROM en_tweets) AND author_id IS NOT NULL
                    GROUP BY author_id;
                    """)
users_f = mycursor.fetchall()
users_f = pd.DataFrame(users_f, columns=[desc[0] for desc in mycursor.description])
print("users class 0:", users_f.shape)

# Adding target variable
users_t['label'] = 1
users_f['label'] = 0

# Concat samples
samples = pd.concat([users_t, users_f], axis=0)
samples= samples.fillna(0)




users class 1 : (15892, 206)
users class 0: (232488, 206)


C:\Users\pelegel\AppData\Local\Temp\ipykernel_18140\732016161.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  samples = pd.concat([users_t, users_f], axis=0)
C:\Users\pelegel\AppData\Local\Temp\ipykernel_18140\732016161.py:30: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  samples= samples.fillna(0)


In [6]:
retweets_all = retweets_all[['author_id', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9']]

grouped_retweets = retweets_all.groupby('author_id').sum().reset_index()
samples = pd.merge(samples, grouped_retweets, left_on='id', right_on='author_id', how='left')
samples.drop('author_id', axis=1, inplace=True)
print(samples)


                         id               id_str                   name  \
0                       324                  324           Chris Fralic   
1                       339                  339              Tony Rose   
2                       544                  544                  Peter   
3                       765                  765        Sean Bonner Ⓥ🛡️   
4                      1497                 1497               harper 🤯   
...                     ...                  ...                    ...   
248375  1609884823602212865  1609884823602212865            Tarun_Kumar   
248376  1609909861898227714  1609909861898227714  Miguel González-Bueno   
248377  1609911768519970817  1609911768519970817              Eve Hough   
248378  1609915175330537474  1609915175330537474              Sweetness   
248379  1609917257383124996  1609917257383124996         Simple n Sweet   

            screen_name           location  \
0           chrisfralic      Phillydelphia   
1      

In [7]:

cols_to_drop = (['id', 'id_str', 'name', 'screen_name', 'location', 'profile_location', 'description', 'entities',
                'created_at', 'utc_offset', 'time_zone', 'lang', 'status', 'translator_type',
                'withheld_in_countries', 'retweets_num'
                 , 'c4_closeness',
                 'a1_in_degree', 'b1_in_degree', 'c1_in_degree',
                 'a2_in_degree', 'b2_in_degree', 'c2_in_degree',
                 'a3_in_degree', 'b3_in_degree', 'c3_in_degree',
                 'a4_in_degree', 'b4_in_degree', 'c4_in_degree',
                 'is_in_a1', 'is_in_a2', 'is_in_a3', 'is_in_a4',
                 'is_in_b1', 'is_in_b2', 'is_in_b3', 'is_in_b4',
                 'is_in_c1', 'is_in_c2', 'is_in_c3', 'is_in_c4'])


samples = samples.drop(columns=cols_to_drop)
y_u = samples['label']
X_u = samples.drop(columns=['label'])

correlations = X_u.corrwith(y_u)
correlations = correlations.abs().sort_values(ascending=False)
print(correlations.head(30))

C:\Users\pelegel\PycharmProjects\pythonProject1\myenv\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\pelegel\PycharmProjects\pythonProject1\myenv\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


neutral_std           0.192784
approval_std          0.155188
admiration_std        0.127515
annoyance_std         0.119979
curiosity_std         0.119149
disapproval_std       0.118155
realization_std       0.116055
disappointment_std    0.113753
neutral_min           0.110237
excitement_std        0.109448
relief_std            0.104936
confusion_std         0.099803
a1_closeness          0.098940
admiration_max        0.098713
a1_out_degree         0.097856
disapproval_max       0.096498
joy_std               0.095871
annoyance_max         0.095415
disappointment_max    0.092698
grief_std             0.089106
pride_std             0.088683
approval_max          0.087618
curiosity_max         0.085564
topic_6               0.085416
listed_count          0.085295
topic_9               0.084355
topic_3               0.082914
topic_5               0.082639
confusion_max         0.078384
amusement_std         0.078035
dtype: float64


In [14]:
print(X_u.columns.to_list())

['protected', 'followers_count', 'friends_count', 'listed_count', 'favourites_count', 'geo_enabled', 'verified', 'statuses_count', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'admiration_avg', 'admiration_min', 'admiration_max', 'admiration_std', 'amusement_avg', 'amusement_min', 'amusement_max', 'amusement_std', 'anger_avg', 'anger_min', 'anger_max', 'anger_std', 'annoyance_avg', 'annoyance_min', 'annoyance_max', 'annoyance_std', 'approval_avg', 'approval_min', 'approval_max', 'approval_std', 'caring_avg', 'caring_min', 'caring_max', 'caring_std', 'confusion_avg', 'confusion_min', 'confusion_max', 'confusion_std', 'curiosity_avg', 'curiosity_min', 'curiosity_max', 'curiosity_std', 'desire_avg', 'desire_min', 'desire_max', 'desire_std', 'disappointment_avg', 'disappointment_min', 'disappointment_max', 'disappointment_std', 'disapproval_avg', 'd

In [8]:
X_u.to_parquet('X_u_tm_1.parquet', index=False)
y_u_df = pd.DataFrame(y_u)
y_u_df.to_parquet('y_u_tm_1.parquet', index=False)